In [ ]:
!pip install optuna
!pip install openpyxl
!pip install imblearn
!pip install xgboost
!pip install transformers
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 69.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 27.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 33.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 10.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 90.8 MB/s eta

In [ ]:
import pandas as pd
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D,GlobalMaxPooling1D, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import ast
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
import transformers
from transformers import AutoModel, BertTokenizerFast
import keras
from sklearn.metrics import balanced_accuracy_score
import keras.backend as K
import numpy

import torch
import torch.nn as nn


2024-06-24 05:31:48.581037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 05:31:48.714824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-24 05:31:48.714855: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-24 05:31:48.746477: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 05:31:49.480796: W tensorflow/stream_executor/pla

In [ ]:
train = pd.read_excel('final_view.xlsx', sheet_name="Sheet1")
test = pd.read_excel('fina_testl_view.xlsx', sheet_name="Sheet1")

In [ ]:
train['final'] = train['final'].apply(lambda x: ast.literal_eval(x))
train['final'] = train['final'].apply(lambda x: ' '.join(x))


test['final'] = test['final'].apply(lambda x: ast.literal_eval(x))
test['final'] = test['final'].apply(lambda x: ' '.join(x))

In [ ]:
train = train[train['final'] != '']
test = test[test['final'] != '']

In [ ]:
train['final'][0]

'kunjung prabowo resmi serah proyek bantu air bersih titik prabowo subianto'

In [ ]:
train['label'] = train['label'].apply(lambda x: x if x=='Ekonomi' else 'Bukan Ekonomi')

In [ ]:
encoder = LabelEncoder()
train['label_encoded'] = encoder.fit_transform(train['label'])


labels = train['label_encoded']


In [ ]:
encoder.inverse_transform([0, 1])

array(['Bukan Ekonomi', 'Ekonomi'], dtype=object)

In [ ]:
train.loc[train['label'] == 'Geografi']

,Unnamed: 0,text,label,metadata,cleaned,raw_mentions,hashtag,tokenize,stemmed,final,label_encoded


In [ ]:
train.loc[train['final'] == '']

,Unnamed: 0,text,label,metadata,cleaned,raw_mentions,hashtag,tokenize,stemmed,final,label_encoded


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train['final'], train['label_encoded'], test_size=0.2, random_state=42, stratify=train['label'])

In [ ]:

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()


In [ ]:

sm = SMOTE(random_state=42)
X_train_tfidf, y_train = sm.fit_resample(X_train_tfidf, y_train)

In [ ]:
train['label'].value_counts()

label
Bukan Ekonomi    4625
Ekonomi           366
Name: count, dtype: int64

In [ ]:
svm_model = SVC(kernel = "linear")
svm_model.fit(X_train_tfidf, y_train)


y_pred_svm = svm_model.predict(X_test_tfidf)
print("SVM Model Accuracy:", accuracy_score(y_test, y_pred_svm))

SVM Model Accuracy: 0.9459459459459459


In [ ]:
print("SVM Model Accuracy:", balanced_accuracy_score(y_test, y_pred_svm))

SVM Model Accuracy: 0.8572738838427172


In [ ]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       926
           1       0.60      0.75      0.67        73

    accuracy                           0.95       999
   macro avg       0.79      0.86      0.82       999
weighted avg       0.95      0.95      0.95       999



In [ ]:
xgb = XGBClassifier(n_estimators=1500)
xgb.fit(X_train_tfidf, y_train)
xgb_pred = xgb.predict(X_test_tfidf)
print(f"Accuracy {accuracy_score(y_test, xgb_pred)}")

Accuracy 0.958958958958959


In [ ]:
print("XGB Model Accuracy:", balanced_accuracy_score(y_test, xgb_pred))

XGB Model Accuracy: 0.8516746057575668


Default Balanced Acuracy with xgb 0.5215757334403693

In [ ]:
# print(classification_report(y_test, xgb_pred))

In [ ]:
tokenizer = Tokenizer(num_words=None, lower=True, oov_token='<OOV>')
tokenizer.fit_on_texts(train['final'])

X = tokenizer.texts_to_sequences(train['final'])
X = pad_sequences(X, maxlen=100) 



y = to_categorical(train['label_encoded'],num_classes=8)


In [ ]:
len(tokenizer.word_index)

8633

In [ ]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
X

array([[   0,    0,    0, ...,  900,    4,   48],
       [   0,    0,    0, ...,  348, 3394,  120],
       [   0,    0,    0, ...,   82,  554,  617],
       ...,
       [   0,    0,    0, ...,  866, 3253,  164],
       [   0,    0,    0, ..., 2310,  893, 1698],
       [   0,    0,    0, ...,   30,    4,   48]], dtype=int32)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from imblearn.combine import SMOTETomek

sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='softmax'))  # Adjust the output layer based on the number of classes

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=16, batch_size=128, validation_data=(X_valid, y_valid), verbose=1)


2024-06-24 05:34:23.175301: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-24 05:34:23.176294: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-24 05:34:23.176350: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-345f8878-fb72-4223-91e4-c32f6cff64f7): /proc/driver/nvidia/version does not exist
2024-06-24 05:34:23.177473: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Epoch 1/16


ValueError: in user code:

    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265, in __call__
   

In [ ]:
test['final']

0      orangorang profesional demokrasi negara ngelaw...
1      prabowo hutang negeri hutang negeri indonesia ...
2      kikidaliyo ganjar pranowo beliau sosok mengagu...
3                   prabowo gibran laku sejahtera rakyat
4      gak nyambung junjung aomkmkmkmk membicarakan e...
                             ...                        
995    bikin bangga deh ganjar mahfud alokasi belanja...
996    jokowi pilpres hati rangkul prabowo prabowo ra...
997    sbaiknya si got nga debat dehnga apakalau sera...
998    rembuk musyawarah gaya pimpin ganjar gubernur ...
999    mirage tolak juwono beli prabowo juru bicara t...
Name: final, Length: 1000, dtype: object

In [ ]:
tokenizer = Tokenizer(num_words=None, lower=True, oov_token='<OOV>')
tokenizer.fit_on_texts(train['final'])

X = tokenizer.texts_to_sequences(train['final'])
X = pad_sequences(X, maxlen=100) 



y = to_categorical(train['label_encoded'],num_classes=8)

In [ ]:
sm = SMOTE(random_state=42)
sm.fit_resample(X, y)

(array([[   0,    0,    0, ...,  900,    4,   48],
        [   0,    0,    0, ...,  348, 3394,  120],
        [   0,    0,    0, ...,   82,  554,  617],
        ...,
        [   0,    0,    0, ...,   95,  237,  159],
        [   0,    0,    0, ...,   13,   83,   45],
        [   0,    0,    0, ...,  415,   93,  315]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 1, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1]]))

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='softmax'))  # Adjust the output layer based on the number of classes

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])


history = model.fit(X, y, epochs=16, batch_size=128, verbose=1)


Epoch 1/16
39/39 [==============================] - 49s 1s/step - loss: 1.5754 - accuracy: 0.5732
Epoch 2/16
39/39 [==============================] - 48s 1s/step - loss: 1.3352 - accuracy: 0.5949
Epoch 3/16
39/39 [==============================] - 50s 1s/step - loss: 1.2312 - accuracy: 0.6003
Epoch 4/16
39/39 [==============================] - 48s 1s/step - loss: 0.9931 - accuracy: 0.6900
Epoch 5/16
39/39 [==============================] - 49s 1s/step - loss: 0.7745 - accuracy: 0.7726
Epoch 6/16
39/39 [==============================] - 48s 1s/step - loss: 0.6028 - accuracy: 0.8151
Epoch 7/16
39/39 [==============================] - 49s 1s/step - loss: 0.4919 - accuracy: 0.8509
Epoch 8/16
39/39 [==============================] - 48s 1s/step - loss: 0.4195 - accuracy: 0.8718
Epoch 9/16
39/39 [==============================] - 48s 1s/step - loss: 0.3467 - accuracy: 0.8936
Epoch 10/16
39/39 [==============================] - 50s 1s/step - loss: 0.3115 - accuracy: 0.9030
Epoch 11/16
39/39 [

In [ ]:
X_final = tokenizer.texts_to_sequences(test['final'])
X_final = pad_sequences(X_final, maxlen=100)

In [ ]:
probaaa = model.predict(X_final)
probaaa

32/32 [==============================] - 2s 62ms/step


array([[6.95106428e-05, 1.39353533e-05, 4.89635931e-05, ...,
        9.95308638e-01, 1.02255035e-04, 2.11251572e-05],
       [6.64474283e-05, 1.29158597e-03, 1.77193491e-04, ...,
        1.27795413e-01, 3.48901871e-04, 3.30722309e-03],
       [1.67540985e-03, 5.23553099e-05, 1.56103575e-04, ...,
        2.90193851e-03, 5.89047559e-04, 5.36885520e-04],
       ...,
       [2.07786798e-04, 1.73674559e-03, 2.61765119e-04, ...,
        9.89232421e-01, 2.06773897e-04, 7.45455560e-04],
       [9.52996965e-03, 9.05676544e-01, 3.06442077e-03, ...,
        4.10521552e-02, 2.27063941e-03, 3.27575691e-02],
       [3.39726626e-04, 5.52189630e-03, 4.80378803e-04, ...,
        9.68983948e-01, 7.06402760e-04, 1.97057403e-03]], dtype=float32)

In [ ]:
result = encoder.inverse_transform(probaaa.argmax(axis=1))
result

array(['Politik', 'Pertahanan dan Keamanan', 'Ideologi', 'Politik',
       'Politik', 'Politik', 'Politik', 'Politik', 'Politik', 'Ekonomi',
       'Politik', 'Politik', 'Ideologi', 'Sosial Budaya', 'Politik',
       'Politik', 'Politik', 'Politik', 'Sosial Budaya', 'Politik',
       'Politik', 'Politik', 'Ekonomi', 'Politik', 'Politik', 'Politik',
       'Politik', 'Politik', 'Politik', 'Politik',
       'Pertahanan dan Keamanan', 'Ideologi', 'Politik', 'Politik',
       'Politik', 'Ideologi', 'Politik', 'Politik', 'Politik', 'Politik',
       'Politik', 'Ekonomi', 'Politik', 'Ekonomi', 'Politik', 'Ekonomi',
       'Ekonomi', 'Politik', 'Politik', 'Politik', 'Politik', 'Politik',
       'Politik', 'Politik', 'Politik', 'Politik', 'Politik', 'Politik',
       'Politik', 'Politik', 'Politik', 'Politik', 'Politik',
       'Sosial Budaya', 'Politik', 'Politik', 'Politik', 'Politik',
       'Politik', 'Politik', 'Politik', 'Politik', 'Politik',
       'Pertahanan dan Keamanan', 'Politik', 

In [ ]:
submission = pd.read_csv('submission.csv')
# submission['Kelas'] = result
submission

,IDText,Kelas
0,TXT0001,Politik
1,TXT0002,Politik
2,TXT0003,Politik
3,TXT0004,Politik
4,TXT0005,Politik
...,...,...
995,TXT0996,Politik
996,TXT0997,Politik
997,TXT0998,Politik
998,TXT0999,Politik


In [ ]:
submission.to_csv('final_submission.csv', index=False)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['final']).toarray()
y= train['label_encoded']
X_test = vectorizer.transform(test['final']).toarray()

In [ ]:
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X, y)

In [ ]:
xgb = XGBClassifier(n_estimators=200)
xgb.fit(X, y)
xgb_pred = xgb.predict(X_test)

In [ ]:
submission['Kelas']=encoder.inverse_transform(xgb_pred)
submission.to_csv('xgb_submission.csv', index=False)

In [ ]:
submission

,IDText,Kelas
0,TXT0001,Politik
1,TXT0002,Pertahanan dan Keamanan
2,TXT0003,Ideologi
3,TXT0004,Politik
4,TXT0005,Politik
...,...,...
995,TXT0996,Politik
996,TXT0997,Politik
997,TXT0998,Politik
998,TXT0999,Ekonomi


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=345f8878-fb72-4223-91e4-c32f6cff64f7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>